<a href="https://colab.research.google.com/github/iterat1on/project/blob/main/%08spark_hak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b7888c171905565541fe2f35220e4fd2dacdb8787139f593a2f054ac60e023c1
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
import pyspark as spark
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import glob
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df = pd.read_csv('./drive/MyDrive/final_data.csv',encoding='utf-8')
df

,현장안전센터거리,시간단위기온,시간단위풍속,시간단위가시거리,재산피해액,실효습도,전기사용량(KWh),가스사용량(KWh),화재발생시각_새벽,화재발생시각_저녁,화재발생시각_주간,등급
0,3,0.2,2.2,491,13391,54.273750,18096.210463,11774.100170,1,0,0,3등급
1,2,9.3,1.9,703,35,49.554054,18096.210463,11774.100170,0,0,1,3등급
2,1,-4.2,1.8,2000,308,31.607682,18096.210463,11774.100170,0,1,0,3등급
3,1,-4.4,4.5,558,184,50.355768,18096.210463,11774.100170,0,0,1,3등급
4,3,1.5,5.9,634,877,43.742292,18096.210463,11774.100170,1,0,0,3등급
...,...,...,...,...,...,...,...,...,...,...,...,...
21247,1,7.1,2.2,1773,11,53.538975,1508.576577,2753.428205,0,1,0,3등급
21248,2,-0.3,2.0,2000,4000,48.487317,1508.576577,2753.428205,0,0,1,3등급
21249,1,4.7,0.6,137,58,55.215504,10826.115819,30619.671937,0,0,0,2등급
21250,2,2.3,1.1,950,18,53.850522,10826.115819,30619.671937,0,0,1,3등급


In [21]:
def convert_Int(x) :
  try :
    result = int(x[0])
  except ValueError :
    pass

  return result 
    

In [23]:
df["등급"] = df.등급.apply(lambda x : convert_Int(x))
df

,현장안전센터거리,시간단위기온,시간단위풍속,시간단위가시거리,실효습도,전기사용량(KWh),가스사용량(KWh),화재발생시각_새벽,화재발생시각_저녁,화재발생시각_주간,등급
0,3,0.2,2.2,491,54.273750,18096.210463,11774.100170,1,0,0,3
1,2,9.3,1.9,703,49.554054,18096.210463,11774.100170,0,0,1,3
2,1,-4.2,1.8,2000,31.607682,18096.210463,11774.100170,0,1,0,3
3,1,-4.4,4.5,558,50.355768,18096.210463,11774.100170,0,0,1,3
4,3,1.5,5.9,634,43.742292,18096.210463,11774.100170,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
21247,1,7.1,2.2,1773,53.538975,1508.576577,2753.428205,0,1,0,3
21248,2,-0.3,2.0,2000,48.487317,1508.576577,2753.428205,0,0,1,3
21249,1,4.7,0.6,137,55.215504,10826.115819,30619.671937,0,0,0,2
21250,2,2.3,1.1,950,53.850522,10826.115819,30619.671937,0,0,1,3


In [24]:
#df = pd.read_csv('./drive/MyDrive/final_data.csv',encoding='utf-8')

#df_test = pd.read_csv('./drive/MyDrive/final_test_data.csv',encoding='utf-8')
spark_df = sqlContext.createDataFrame(df)
#spark_df_test = sqlContext.createDataFrame(df_test)

In [25]:
df.head()

,현장안전센터거리,시간단위기온,시간단위풍속,시간단위가시거리,실효습도,전기사용량(KWh),가스사용량(KWh),화재발생시각_새벽,화재발생시각_저녁,화재발생시각_주간,등급
0,3,0.2,2.2,491,54.273750,18096.210463,11774.10017,1,0,0,3
1,2,9.3,1.9,703,49.554054,18096.210463,11774.10017,0,0,1,3
2,1,-4.2,1.8,2000,31.607682,18096.210463,11774.10017,0,1,0,3
3,1,-4.4,4.5,558,50.355768,18096.210463,11774.10017,0,0,1,3
4,3,1.5,5.9,634,43.742292,18096.210463,11774.10017,1,0,0,3


In [26]:
from pyspark.ml.feature import VectorAssembler

# 1. Train/Test 데이터로 분할
train_df, test_df = spark_df.randomSplit(weights=[0.8, 0.2], seed=42)
#train_df = spark_df
#test_df = spark_df_test


# 2. Feature Vector로 만들기
ftr_columns = ['현장안전센터거리', '시간단위기온', '시간단위풍속', '시간단위가시거리', '실효습도',
       '전기사용량(KWh)', '가스사용량(KWh)', '화재발생시각_새벽', '화재발생시각_저녁', '화재발생시각_주간']
vec_assembler = VectorAssembler(inputCols=ftr_columns, outputCol='features') #features에는 하나의 칼럼.

train_ftr_vec = vec_assembler.transform(train_df)
test_ftr_vec = vec_assembler.transform(test_df)  # 테스트 데이터에 대해도 동일한 객체 사용
sparktrain_ftr_vec = vec_assembler.transform(test_df)

print('# 학습 원본 데이터:')
train_df.limit(5).show()

print('# Feature Vectorization 후 학습 데이터:')
train_ftr_vec.limit(5).show()

# 학습 원본 데이터:
+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|          실효습도|   전기사용량(KWh)|   가스사용량(KWh)|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|등급|
+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+
|               0|       -14.4|         2.0|            2000|         44.281887| 9949.277379066085|21762.540509124723|                0|                0|                0|   2|
|               0|       -12.7|         2.8|            2000| 53.32868400000001| 6579.535143847995| 7826.339856957087|                0|                0|                0|   2|
|               0|       -10.6|         1.7|            2000|53.760867000000005|  9339.12905468026|13023.231325301203|                1|                0|           

#랜덤포레스트

In [27]:
from pyspark.ml.classification import RandomForestClassifier

# 모델 정의
dt_clf = RandomForestClassifier(featuresCol='features', labelCol='등급', numTrees=10)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)


In [28]:
train_pred.show()

+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|          실효습도|   전기사용량(KWh)|   가스사용량(KWh)|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|등급|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|               0|       -14.4|         2.0|            2000|         44.281887| 9949.277379066085|21762.540509124723|                0|                0|                0|   2|[0.0,-14.4,2.0,20...|[0.0,0.0675161825...|[0.0,0.0067516182...|       2.0|
|               0|       -12.7|         2.8|            2000

In [29]:
test_pred.show()

+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|          실효습도|   전기사용량(KWh)|   가스사용량(KWh)|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|등급|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|               0|       -10.6|         1.7|            2000|         38.341068|14438.628358501666|29561.960814249363|                0|                0|                1|   3|[0.0,-10.6,1.7,20...|[0.0,0.0017305800...|[0.0,1.7305800110...|       3.0|
|               0|        -9.6|         1.1|            1748

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='등급', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
#test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
#print('Test Accuracy:', test_acc)

Train Accuracy: 0.9888992755316662


#결정트리

In [32]:
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 정의
dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='등급', maxDepth=10)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='등급', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
#test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
#print('Test Accuracy:', test_acc)

Train Accuracy: 0.9992404767469035


In [35]:
test_pred.show()

+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|          실효습도|   전기사용량(KWh)|   가스사용량(KWh)|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|등급|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+----------+
|               0|       -10.6|         1.7|            2000|         38.341068|14438.628358501666|29561.960814249363|                0|                0|                1|   3|[0.0,-10.6,1.7,20...|[0.0,0.0,0.0,9040.0]|   [0.0,0.0,0.0,1.0]|       3.0|
|               0|        -9.6|         1.1|            1748